## Bayesian Unobserved Components model example

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import statsmodels.api as sm
import theano
import theano.tensor as tt
from pandas.plotting import register_matplotlib_converters
from pandas_datareader.data import DataReader

register_matplotlib_converters()

In [3]:
# data = DataReader('^DJI', 'stooq', start="1990-01", end="2024-02")
# data[:10]
# print(data.shape)
# data = data.dropna()
# print(data.tail())
# print(data.head())

Example data: US Consumer Price Index

In [ ]:
cpi = DataReader("CPIAUCNS", "fred", start="1971-01", end="2024-02")  #  FED: Consumer Price Index for All Urban Consumers: All Items
cpi.index = pd.DatetimeIndex(cpi.index, freq="MS")

inf = np.log(cpi).resample("QS").mean().diff()[1:] * 400
inf = inf.dropna()

print(inf.shape)
print(inf.tail())
print(inf.head())

In [ ]:
# Plot data
fig, ax = plt.subplots(figsize=(8, 3), dpi=200)
ax.plot(inf.index, inf, label=r"$\Delta \log CPI$", lw=1)
ax.legend(loc="lower left")
plt.title("US Consumer Price Index for All Urban Consumers")
plt.show()

In [6]:
# Create an SARIMAX model instance - here we use it to estimate
# the parameters via MLE using the `fit` method, but we can
# also re-use it below for the Bayesian estimation
mod = sm.tsa.statespace.SARIMAX(inf, order=(1, 0, 1))

res_mle = mod.fit(disp=False)
#print(res_mle.summary())

In [ ]:
predict_mle = res_mle.get_prediction()
predict_mle_ci = predict_mle.conf_int()
lower = predict_mle_ci["lower CPIAUCNS"]
upper = predict_mle_ci["upper CPIAUCNS"]

# Graph
fig, ax = plt.subplots(figsize=(9, 4), dpi=300)

# Plot data points
inf.plot(ax=ax, style="-", label="Observed")

# Plot predictions
predict_mle.predicted_mean.plot(ax=ax, style="r.", label="One-step-ahead forecast")
ax.fill_between(predict_mle_ci.index, lower, upper, color="r", alpha=0.1)
ax.legend(loc="lower left")
plt.show()

Construct loglikelihood and score function for pymc:

In [67]:
class Loglike(tt.Op):

    itypes = [tt.dvector]  # expects a vector of parameter values when called
    otypes = [tt.dscalar]  # outputs a single scalar value (the log likelihood)

    def __init__(self, model):
        self.model = model
        self.score = Score(self.model)

    def perform(self, node, inputs, outputs):
        (theta,) = inputs  # contains the vector of parameters
        llf = self.model.loglike(theta)
        outputs[0][0] = np.array(llf)  # output the log-likelihood

    def grad(self, inputs, g):
        # the method that calculates the gradients - it actually returns the
        # vector-Jacobian product - g[0] is a vector of parameter values
        (theta,) = inputs  # our parameters
        out = [g[0] * self.score(theta)]
        return out


class Score(tt.Op):
    itypes = [tt.dvector]
    otypes = [tt.dvector]

    def __init__(self, model):
        self.model = model

    def perform(self, node, inputs, outputs):
        (theta,) = inputs
        outputs[0][0] = self.model.score(theta)

In [68]:
# Set sampling params
ndraws = 3000  # number of draws from the distribution
nburn = 600    # number of "burn-in points" (which will be discarded)

In [ ]:
# Construct an instance of the Theano wrapper defined above, which
# will allow PyMC3 to compute the likelihood and Jacobian in a way
# that it can make use of. Here we are using the same model instance
# created earlier for MLE analysis (we could also create a new model
# instance if we preferred)
loglike = Loglike(mod)

with pm.Model() as m:
    # Priors
    arL1 = pm.Uniform("ar.L1", -0.99, 0.99)
    maL1 = pm.Uniform("ma.L1", -0.99, 0.99)
    sigma2 = pm.InverseGamma("sigma2", 2, 4)

    # convert variables to tensor vectors
    theta = tt.as_tensor_variable([arL1, maL1, sigma2])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist("likelihood", loglike, observed=theta)

    # Draw samples
    trace = pm.sample(
        ndraws,
        tune=nburn,
        return_inferencedata=True,
        cores=1,
        compute_convergence_checks=False,
    )

In [ ]:
plt.tight_layout()
# Note: the syntax here for the lines argument is required for
# PyMC3 versions >= 3.7
# For version <= 3.6 you can use lines=dict(res_mle.params) instead
_ = pm.plot_trace(
    trace,
    lines=[(k, {}, [v]) for k, v in dict(res_mle.params).items()],
    combined=True,
    figsize=(12, 12),
)

In [ ]:
pm.summary(trace)

In [ ]:
# Retrieve the posterior means
params = pm.summary(trace)["mean"].values

# Construct results using these posterior means as parameter values
res_bayes = mod.smooth(params)

predict_bayes = res_bayes.get_prediction()
predict_bayes_ci = predict_bayes.conf_int()
lower = predict_bayes_ci["lower CPIAUCNS"]
upper = predict_bayes_ci["upper CPIAUCNS"]

# Graph
fig, ax = plt.subplots(figsize=(9, 4), dpi=300)

# Plot data points
inf.plot(ax=ax, style="-", label="Observed")

# Plot predictions
predict_bayes.predicted_mean.plot(ax=ax, style="r.", label="One-step-ahead forecast")
ax.fill_between(predict_bayes_ci.index, lower, upper, color="r", alpha=0.1)
ax.legend(loc="lower left")
plt.show()

In [ ]:
# Construct the model instance
mod_uc = sm.tsa.UnobservedComponents(inf, "rwalk", autoregressive=1)

# Fit the model via maximum likelihood
res_uc_mle = mod_uc.fit()
print(res_uc_mle.summary())

In [74]:
# Set sampling params
ndraws = 3000  # number of draws from the distribution
nburn = 600  # number of "burn-in points" (which will be discarded)

In [ ]:
# Here we follow the same procedure as above, but now we instantiate the
# Theano wrapper `Loglike` with the UC model instance instead of the
# SARIMAX model instance
loglike_uc = Loglike(mod_uc)

with pm.Model():
    # Priors
    sigma2level = pm.InverseGamma("sigma2.level", 1, 1)
    sigma2ar = pm.InverseGamma("sigma2.ar", 1, 1)
    arL1 = pm.Uniform("ar.L1", -0.99, 0.99)

    # convert variables to tensor vectors
    theta_uc = tt.as_tensor_variable([sigma2level, sigma2ar, arL1])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist("likelihood", loglike_uc, observed=theta_uc)

    # Draw samples
    trace_uc = pm.sample(
        ndraws,
        tune=nburn,
        return_inferencedata=True,
        cores=1,
        compute_convergence_checks=False,
    )

In [ ]:
plt.tight_layout()
# Note: the syntax here for the lines argument is required for
# PyMC3 versions >= 3.7
# For version <= 3.6 you can use lines=dict(res_mle.params) instead
_ = pm.plot_trace(
    trace_uc,
    lines=[(k, {}, [v]) for k, v in dict(res_uc_mle.params).items()],
    combined=True,
    figsize=(12, 12),
)

In [ ]:
pm.summary(trace_uc)

In [78]:
# Retrieve the posterior means
params = pm.summary(trace_uc)["mean"].values

# Construct results using these posterior means as parameter values
res_uc_bayes = mod_uc.smooth(params)

In [79]:
predict_bayes = res_uc_bayes.get_prediction()
predict_bayes_ci = predict_bayes.conf_int()

lower = predict_bayes_ci["lower CPIAUCNS"]
upper = predict_bayes_ci["upper CPIAUCNS"]

In [ ]:
# Graph
fig, ax = plt.subplots(figsize=(9, 4), dpi=300)

# Plot data points
inf["CPIAUCNS"].plot(ax=ax, style="-", label="Observed data")

# Plot estimate of the level term
res_uc_mle.states.smoothed["level"].plot(ax=ax, label="Smoothed level (MLE)")
res_uc_bayes.states.smoothed["level"].plot(ax=ax, label="Smoothed level (Bayesian)")

ax.legend(loc="lower left");

# AR(1):

In [ ]:
import matplotlib.pyplot as plt
#from mpl_toolkits.mplot3d import Axes3D
#from scipy.stats import wishart, multivariate_normal, bernoulli, multinomial
#from scipy.sparse import csr_matrix
#from sklearn.model_selection import train_test_split
import os, pickle
import numpy as np
import math
#from numpy import log, sum, exp, prod
from numpy.random import beta, binomial, normal, dirichlet, uniform, gamma, seed, multinomial, gumbel, rand
from imp import reload
from copy import deepcopy
from math import cos, pi
#import seaborn as sns
import pandas as pd
import time
#from scipy.spatial.distance import euclidean
import itertools
from itertools import chain, combinations


def cycle(N, nof_cycles = 1):
  return np.cos(2*pi*np.arange(0,N)*nof_cycles/N)

def simAR1(N, phi, sigma, const = 0, burn=100):
  y = np.zeros((N+burn))
  for t in range(N+burn-1):
    y[t+1] = const + phi*y[t] + normal(scale = sigma, size=1)
  return y[burn:]

#np.random.seed(0)   # set seed

N = 2*10**3
omega = 1
phi_true = 0.77
sigma_true = 0.5

y = simAR1(N, phi = phi_true, sigma = sigma_true, const = 0.5)
#y1 = omega*cycle(N, nof_cycles = 2) + simAR1(N, phi = 0.7, sigma = 0.6)
#y2 = omega*cycle(N, nof_cycles = 2) + simAR1(N, phi = 0.7, sigma = 0.6)
#y3 = omega*cycle(N , nof_cycles = 2) + simAR1(N , phi = 0.5, sigma = 1.4)
#y4 = omega*cycle(N, nof_cycles = 2)

# Plot trajectories:
#---------------------
plt.figure()
pd.Series(y).plot()
plt.show()

In [ ]:
# Construct the model instance
mod_ar1 = sm.tsa.statespace.SARIMAX(y, trend = 'c', order=(1,0,0), seasonal_order=(0,0,0,0))

# Fit the model via maximum likelihood
res_ar_mle = mod_ar1.fit()
print(res_ar_mle.summary())

In [ ]:
loglike = Loglike(mod_ar1)

#ndraws = 3000

with pm.Model() as model:

    # Step 1: Define statistical model symbolically, prior and likelihood

    # Priors
    arL1 = pm.Uniform("ar.L1", -0.99, 0.99)
    intercept = pm.Uniform("intercept", -2, 2)
    sigma2 = pm.InverseGamma("sigma2", 2, 4)

    # set predictors as shared variable to change them for PPCs:
    #pred = pm.Data("pred", y)

    # convert variables to tensor vectors
    theta = tt.as_tensor_variable([arL1, intercept, sigma2])

    # use a DensityDist (use a lamdba function to "call" the Op)
    pm.DensityDist("likelihood", loglike, observed=theta)

    # Step 2: Inference step. Define algorithm for learning and prediction

    # MCMC inference:
    start = None #pm.find_MAP()
    inference = pm.NUTS()
    trace = pm.sample(ndraws, inference, start=start, tune=nburn,
                      return_inferencedata=True, compute_convergence_checks=False)

    # fit model via variational inference:
    #inference = pm.ADVI()                   # Note, that this is a mean-field approximation so we ignore correlations in the posterior.
    #inference = pm.SVGD(n_particles=500, jitter=1)

    # tracker = pm.callbacks.Tracker(
    #         mean = inference.approx.mean.eval,  # callable that returns mean
    #         std = inference.approx.std.eval  # callable that returns std
    # )

    # approx = pm.fit(n=ndraws, method = inference, callbacks=[tracker])      # n: number of iterations
    # trace = approx.sample(ndraws, inference)    # sample from the variational distr.


    # Draw samples
    # trace = pm.sample(
    #     ndraws,
    #     tune=nburn,
    #     return_inferencedata=True,
    #     cores=1,
    #     compute_convergence_checks=False,
    # )

In [ ]:
def predict_outofsample(trace, date_idx):
        """
        trace: a pymc3 MultiTrace object
        date_idx: np.ndarray with shape (N_obs), indicating for each observation what date it corresponds to
            (so you can have multiple observations on the same day that will have the same prediction)
        """
        samples = []
        horizon = np.max(date_idx)
        for point in enumerate(trace.points()):
            rho, scale = point['rho'], point['scale']
            thetas = [np.random.normal(loc=0, scale=scale)]
            for i in range(horizon):
                thetas.append(rho*thetas[-1] + np.random.normal(loc=0, scale=scale))
            samples.append(thetas)
        return np.array(samples)[:, date_idx]

In [ ]:
#dir(model)

In [ ]:
#waic = pm.waic(trace, model)
#print(waic)

In [ ]:
# Plotting the objective function (ELBO) we can see that the optimization slowly improves the fit over time.
plt.plot(-inference.hist, label='new ADVI', alpha=.3)
plt.plot(approx.hist, label='old ADVI', alpha=.3)
plt.legend()
plt.ylabel('ELBO')
plt.xlabel('iteration');

In [ ]:
names = ['arL1', 'intercept', 'sigma2']
dict_paras = {n:v for n,v in zip(names, res_ar_mle.params)}

In [ ]:
plt.tight_layout()

_ = pm.plot_trace(
    trace,
    lines=[(k, {}, [v]) for k, v in dict_paras.items()],
    combined=True,
    figsize=(12, 12),
)

In [ ]:
pm.summary(trace)

In [ ]:
trace

In [ ]:
# Retrieve the posterior means
params = pm.summary(trace)["mean"].values

# Construct results using these posterior means as parameter values
res_bayes = mod_ar1.smooth(params)

predict_bayes = res_bayes.get_prediction()
predict_bayes_ci = predict_bayes.conf_int()
lower = predict_bayes_ci["lower CPIAUCNS"]
upper = predict_bayes_ci["upper CPIAUCNS"]

# Graph
fig, ax = plt.subplots(figsize=(9, 4), dpi=300)

# Plot data points
inf.plot(ax=ax, style="-", label="Observed")

# Plot predictions
predict_bayes.predicted_mean.plot(ax=ax, style="r.", label="One-step-ahead forecast")
ax.fill_between(predict_bayes_ci.index, lower, upper, color="r", alpha=0.1)
ax.legend(loc="lower left")
plt.show()

In [ ]:
trace['posterior']['chain'][0].shape